In [31]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup
import nltk
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
base_filemask = '/data/share/lab05data/base_{}.txt'
base_df = pd.DataFrame(columns=['id', 'text'])
for i in range(1, 21):
    base_filename = base_filemask.format(i)
    with open(base_filename, 'r') as file:
        base_df = base_df.append({'id': i, 'text': re.sub('<[^<]+?>', '', str(file.readline()))}, ignore_index=True)

In [100]:
test_filemask = '/data/share/lab05data/test_{}.txt'
test_df = pd.DataFrame(columns=['id', 'text'])
for i in (2049, 3587, 3588, 5, 3078, 2570, 3600, 2579, 4, 533, 3607, 1560, 3610, 2076, 30, 2591, 2457, 2084, 
          776, 46, 1586, 3123, 3125, 3639, 2621, 2622, 70, 2634, 587, 588, 2126, 1616, 3668, 1622, 88, 2649, 
          869, 3175, 1632, 1121, 1126, 1639, 189, 1644, 1647, 2238, 1651, 630, 3708, 3198, 3713, 1154, 3203, 
          2668, 1669, 2183, 1674, 3728, 2194, 147, 665, 1690, 624, 2206, 672, 1188, 678, 1626, 1704, 2731, 173, 
          1054, 180, 2230, 695, 3256, 1211, 701, 2411, 1728, 1217, 3780, 2758, 3783, 2253, 2763, 1741, 2255, 
          1150, 1747, 213, 3292, 3296, 636, 2280, 3265, 1406, 241, 3317, 758, 3833, 2816, 1281, 3332, 775, 1800, 
          1289, 3850, 2828, 783, 2323, 1305, 3420, 3376, 1311, 800, 1829, 2855, 808, 2861, 2864, 2353, 1331, 3892, 
          3551, 3722, 3387, 3808, 2878, 1344, 833, 2884, 2374, 3913, 336, 1361, 2389, 1367, 656, 1885, 2398, 1892, 
          357, 2406, 2450, 1387, 3438, 1911, 382, 1407, 1345, 2438, 1419, 1932, 909, 399, 913, 402, 917, 406, 409, 
          3824, 414, 1439, 3489, 2467, 421, 2984, 1961, 427, 3959, 1967, 3067, 2463, 3001, 955, 958, 715, 972, 978, 
          562, 3544, 3547, 1500, 3038, 479, 2215, 2535, 3562, 3564, 1006, 2544, 3570, 1013, 887, 3066, 2043, 3580, 510, 2134):
    test_filename = test_filemask.format(i)
    with open(test_filename, 'r') as file:
        test_df = test_df.append({'id': i, 'text': re.sub('<[^<]+?>', '', str(file.readline()))}, ignore_index=True)

In [119]:
import pymorphy2
m = pymorphy2.MorphAnalyzer()

GROUPING_SPACE_REGEX = re.compile(r'([^\w]|[+])', re.UNICODE)
def simple_word_tokenize(text, _split=GROUPING_SPACE_REGEX.split):
    return(t for t in _split(text.lower()) if t and not t.isspace())

def token_r(text):
    words = simple_word_tokenize(text)
    return [m.parse(x)[0].normal_form for x in words if len(x)>=4]

cv = CountVectorizer(tokenizer=token_r)

In [120]:
matrix_base = cv.fit_transform(base_df['text'])
matrix_test = cv.transform(test_df['text'])

cos_m = cosine_similarity(matrix_base,matrix_test)
sum_cos = np.sum(cos_m, axis=0)
avg_cos = np.sum(sum_cos)/200

In [104]:
sum_cos_df = pd.DataFrame(sum_cos)
test_df['sum_cos'] = sum_cos_df[0]

In [109]:
test_df['result'] = np.where(test_df['sum_cos']>=avg_cos,'1','0')
sum_cos = np.sum(cos_m, axis=0)
avg_cos = np.sum(sum_cos)/200
result_json = {'defined': defined, 'other': other}
import json
with open('lab05.json', 'w') as outfile:
    json.dump(result_json, outfile)